In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
import scanpy as sc
import scvelo as scv
import pandas as pd
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch import nn

import model, data
import os

os.environ["DGLBACKEND"] = "pytorch"

In [2]:
scrna_path = "chicken_heart\\RNA_D14_adata.h5ad"
st_path = "chicken_heart\\Visium_D14_adata.h5ad"

scrna = scv.read(scrna_path)

In [3]:
scrna.obs

,orig.ident,nCount_RNA,nFeature_RNA,percent.mito,S.Score,G2M.Score,Phase,RNA_snn_res.0.5,seurat_clusters,scanorama_snn_res.0.5,scanorama_snn_res.0.4,scanorama_snn_res.0.3,scanorama_snn_res.0.25,scanorama_snn_res.0.2,day,ventricle,celltypes.0.5,n_counts
AAACCTGCATTGTGCA,D14-LV,1163,536,14.273431,-0.383406,-0.500917,G1,0,11,11,1,0,1,1,D14,LV,Mural cells,9.999959e+05
AAACGGGAGAAGGTTT,D14-LV,575,338,0.869565,0.099729,-0.011873,S,0,6,6,5,9,5,2,D14,LV,TMSB4X high cells,9.999998e+05
AAACGGGAGGCCATAG,D14-LV,574,334,14.808362,-0.021670,-0.388788,G1,0,0,0,1,0,1,1,D14,LV,Fibroblast cells,9.999979e+05
AAACGGGAGTTACCCA,D14-LV,550,296,7.818182,-0.256206,-0.097799,G1,0,0,0,1,0,1,1,D14,LV,Fibroblast cells,9.999970e+05
AAACGGGCACGGCCAT,D14-LV,584,331,10.958904,0.163577,-0.065646,S,6,6,6,5,9,5,2,D14,LV,TMSB4X high cells,1.000001e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTGTGCGAAAC,D14-RV,1041,472,13.544669,0.196279,0.197302,G2M,6,9,9,9,10,9,8,D14,RV,Vascular endothelial cells,9.999956e+05
TTTGGTTGTGCTGTAT,D14-RV,746,349,14.343164,-0.255970,-0.160041,G1,0,0,0,1,0,1,1,D14,RV,Fibroblast cells,1.000002e+06
TTTGGTTTCGCAAGCC,D14-RV,586,318,6.825939,-0.222434,-0.337061,G1,0,0,0,1,0,1,1,D14,RV,Fibroblast cells,1.000002e+06
TTTGTCAAGTAACCCT,D14-RV,2766,1007,0.108460,-0.707029,1.184469,G2M,4,2,2,2,2,2,0,D14,RV,Immature myocardial cells,9.999992e+05


In [5]:
data = data.SingleCellDataset(st_path)

Filtered out 10078 genes that are detected 20 counts (shared).
Extracted 2000 highly variable genes.
Logarithmized X.


c:\Users\Tian\AppData\Local\Programs\Python\Python311\Lib\site-packages\scvelo\preprocessing\utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing neighbors
    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [6]:
st = scv.read(st_path)